In [10]:
import numpy as np
from tool.utils import *
from models import *
import cv2
import torch
from torch import nn
import torch.nn.functional as F
import copy
from cfg import Cfg

In [11]:
model = Yolov4(n_classes=2)
pretrained_dict = torch.load('checkpoints/Yolov4_epoch600.pth',map_location=torch.device('cuda'))
model.load_state_dict(pretrained_dict)
model.cuda()

Yolov4(
  (down1): DownSample1(
    (conv1): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv2): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv3): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Mish()
      )
    )
    (conv4): Conv_Bn_Activation(
      (conv): ModuleList(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNo

In [12]:
class_names = ['O', 'X']

In [13]:
def predictions(file, conf_thresh, path):
    cnt = 1
    truth = {}
    for line in file.readlines():
        data = line.split(" ")
        truth[data[0]] = [] 
        for i in data[1:]: 
            truth[data[0]].append([int(j) for j in i.split(',')]) 
        gts=truth[data[0]] 
        img = cv2.imread(path+data[0])
        print(data[0])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        sized = cv2.resize(img, (Cfg.height, Cfg.width))
        boxes = do_detect(model, sized, conf_thresh, 2, 0.4, 1)
        plot_boxes_cv2(img, boxes, 'predictions.jpg', class_names) #畫出預測框
        img = cv2.imread('predictions.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        for gt in gts:
            x1 = gt[0]
            y1 = gt[1]
            x2 = gt[2]
            y2 = gt[3]
            cls_id = gt[4]
            img = cv2.putText(img, class_names[cls_id], (x1+(x2-x1)//2, y1), 
                              cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 2)
            img = cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,0), 10)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        savename = 'prediction_imgs/'+'pred'+str(cnt)+'.jpg'
        cv2.imwrite(savename, img)
        cnt += 1

In [14]:
file_name = 'data/test3.txt'
file = open(file_name, 'r', encoding='utf-8')
conf_thresh = 0.5
predictions(file, conf_thresh, 'testset3/')
print('Running End')

2f5348c8-8518-476b-a204-576670867e53.jpg
O: 0.999598
486 251 759 1171
O: 0.993488
751 485 1224 1341
O: 0.999471
181 316 492 1136
save plot results to predictions.jpg
000126.jpg
O: 0.984010
425 382 732 682
O: 0.966078
1265 393 1517 656
O: 0.880971
957 394 1185 680
save plot results to predictions.jpg
P_20210128_141654.jpg
save plot results to predictions.jpg
000163.jpg
O: 0.987852
1521 1253 2066 1864
O: 0.995924
1042 1291 1531 1869
save plot results to predictions.jpg
P_20210128_141413.jpg
O: 0.567392
2808 1118 3244 2078
save plot results to predictions.jpg
000122.jpg
O: 0.999838
357 314 633 665
O: 0.999021
888 322 1203 718
save plot results to predictions.jpg
P_20210128_141600.jpg
save plot results to predictions.jpg
P_20210329_165209.jpg
X: 0.997338
1144 596 1801 1765
X: 0.999990
2631 787 3467 1986
X: 0.997874
601 542 1248 1826
save plot results to predictions.jpg
000161.jpg
O: 0.996837
257 328 597 571
save plot results to predictions.jpg
P_20210128_141525.jpg
X: 0.770067
1964 1048 24

In [7]:
file_name = 'train.txt'
file = open(file_name, 'r', encoding='utf-8') 
conf_thresh = 0.5
predictions(file, conf_thresh, 'train/')
print('Running End')

000191.jpg


TypeError: can't assign a numpy.float128 to a torch.FloatTensor

In [6]:
!zip -r imgs.zip prediction_imgs

updating: prediction_imgs/ (stored 0%)
updating: prediction_imgs/pred85.jpg (deflated 1%)
updating: prediction_imgs/pred46.jpg (deflated 0%)
updating: prediction_imgs/pred39.jpg (deflated 7%)
updating: prediction_imgs/pred94.jpg (deflated 0%)
updating: prediction_imgs/pred26.jpg (deflated 0%)
updating: prediction_imgs/pred124.jpg (deflated 0%)
updating: prediction_imgs/pred96.jpg (deflated 3%)
updating: prediction_imgs/pred100.jpg (deflated 0%)
updating: prediction_imgs/pred76.jpg (deflated 3%)
updating: prediction_imgs/pred35.jpg (deflated 1%)
updating: prediction_imgs/pred4.jpg (deflated 12%)
updating: prediction_imgs/pred95.jpg (deflated 0%)
updating: prediction_imgs/pred37.jpg (deflated 4%)
updating: prediction_imgs/pred30.jpg (deflated 4%)
updating: prediction_imgs/pred123.jpg (deflated 2%)
updating: prediction_imgs/pred42.jpg (deflated 0%)
updating: prediction_imgs/pred111.jpg (deflated 2%)
updating: prediction_imgs/pred69.jpg (deflated 0%)
updating: prediction_imgs/pred102.jpg (